# Imports

This Lab uses numpy, matplotlib, pillow, pandas, scipy, scikit-learn and scikit-image.

In [ ]:
#imports
import pandas as pd
print(1)

1


In [ ]:
df=pd.read_csv("spotify_history.csv")
arrShuffle=df["shuffle"]
arrReason_End=df["reason_end"]
arrMs_Played=df["ms_played"]
arrReason_Start=df["reason_start"]

i=0
print(len(arrReason_End))
for x in range(len(arrReason_End)):
    if(arrReason_End[x]=="trackdone"):
        arrReason_End[x]=1
        i+=1
    else:
        arrReason_End[x]=0
print(i)


C:\Users\maxim\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\maxim\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


77194
